# ⚡ Performance Optimization & Tuning

**Phase 5: Performance Mastery - Optimizing Spark for Speed & Scale**

**Prerequisites**: [04_spark_sql/Spark_SQL_Practice.ipynb](../04_spark_sql/Spark_SQL_Practice.ipynb)

**Estimated time**: 60-75 minutes

---

## 🎯 Learning Goals

By the end of this notebook, you'll be able to:
- ✅ Understand and minimize shuffle operations
- ✅ Optimize data partitioning strategies
- ✅ Use caching and persistence effectively
- ✅ Tune Spark configuration for performance
- ✅ Identify and resolve data skew issues
- ✅ Monitor and troubleshoot slow jobs

---

## ⚙️ Setup & Performance Monitoring

**Initialize Spark and set up performance monitoring.**

In [ ]:
// Import Spark libraries
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.functions._
import spark.implicits._

In [ ]:
// Create SparkSession with performance monitoring
val spark = SparkSession.builder()
  .appName("Performance Tuning Practice")
  .master("local[*]")
  .config("spark.sql.adaptive.enabled", "true")
  .config("spark.sql.adaptive.coalescePartitions.enabled", "true")
  .getOrCreate()

val sc = spark.sparkContext

println("🚀 Performance Tuning Practice Started")
println(s"Available cores: ${sc.defaultParallelism}")
println(s"Default parallelism: ${sc.defaultParallelism}")

In [ ]:
// Performance monitoring function
def timeOperation[T](operationName: String)(block: => T): T = {
  val startTime = System.nanoTime()
  val result = block
  val endTime = System.nanoTime()
  val durationMs = (endTime - startTime) / 1e6
  println(f"⚡ $operationName completed in $durationMs%.2f ms")
  result
}

// Create large test dataset
val largeDataset = sc.parallelize(1 to 100000).map(i => (s"Key${i % 100}", i))
println(s"Created dataset with ${largeDataset.count()} records")
println(s"Partitions: ${largeDataset.getNumPartitions}")

## 🔀 Understanding Shuffle Operations

**Shuffles are expensive - learn to minimize them!**

In [ ]:
// Demonstrate shuffle operations
println("🔀 Shuffle Operations Analysis:")

// Operations that cause shuffles
val data = sc.parallelize(Seq(
  ("A", 1), ("B", 2), ("A", 3), ("B", 4), ("C", 5)
))

println("Original data:")
data.collect().foreach(println)

// groupByKey - CAUSES SHUFFLE
val grouped = timeOperation("groupByKey") {
  data.groupByKey().collect()
}
println("groupByKey result:")
grouped.foreach { case (k, v) => println(s"$k -> ${v.mkString(",")}") }

// reduceByKey - MORE EFFICIENT (combines locally first)
val reduced = timeOperation("reduceByKey") {
  data.reduceByKey(_ + _).collect()
}
println("\nreduceByKey result:")
reduced.foreach(println)

## 📦 Caching & Persistence

**Cache frequently used data to avoid recomputation.**

In [ ]:
// Demonstrate caching benefits
println("📦 Caching & Persistence:")

val expensiveComputation = sc.parallelize(1 to 10000)
  .map(x => (x % 100, x))
  .reduceByKey(_ + _)
  .filter(_._2 > 50000)

// First access - no caching
val firstAccess = timeOperation("First access (no cache)") {
  expensiveComputation.collect()
}
println(s"First access result count: ${firstAccess.length}")

// Cache the RDD
expensiveComputation.cache()
println("\n📦 RDD cached in memory")

// Second access - from cache
val secondAccess = timeOperation("Second access (from cache)") {
  expensiveComputation.collect()
}
println(s"Second access result count: ${secondAccess.length}")

println("\n💡 Caching benefits:")
println("   - Avoids recomputation of expensive operations")
println("   - Speeds up iterative algorithms")
println("   - Useful for frequently accessed data")

## 🏆 Performance Optimization Exercises

**Apply performance tuning techniques to optimize Spark jobs.**

In [ ]:
// Exercise 1: Optimize Word Count
// FIXME: Optimize this word count implementation

println("📝 Performance Optimization Exercise:")

val text = sc.parallelize(Seq(
  "hello world hello spark",
  "world spark is great"
) * 50)

// Original implementation
val original = timeOperation("Original word count") {
  text.flatMap(_.split(" "))
      .map(word => (word, 1))
      .reduceByKey(_ + _)
      .collect()
}

// FIXME: Create optimized version
val optimized = timeOperation("Optimized word count") {
  // Your optimized implementation here
  original // Placeholder
}

println("\n💡 Optimizations to consider:")
println("   - Use coalesce() to reduce partitions")
println("   - Filter empty strings")
println("   - Case normalization")

## 🛑 Cleanup

**Clean up cached data and stop SparkSession.**

In [ ]:
// Stop SparkSession
spark.stop()
println("🛑 Performance Tuning Session Stopped")
println("✅ All resources cleaned up!")

## 📚 What Next?

**🎉 Congratulations!** You've mastered Spark performance optimization!

**Key Concepts:**
- ✅ Shuffle operations and their costs
- ✅ Caching and persistence strategies
- ✅ Data partitioning and skew handling

**Next:** Move to **06_streaming/** for real-time processing!

**Remember:** Performance optimization is critical for production Spark jobs! ⚡